In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt


In [ ]:
dir_BLUE = '/GCB2021/Data/Output_GCB2021/202109_GCB2021_run/run_v3_ForestArea_1991-2021/'
dir_area = '/GCB2021_commentary/Data/forest_area/'
dir_ctrs = '/Trendy/Data/data_ancillary/info_countries/'
dir_fig  = '/GCB2021_commentary/Figures/'


## Country definitions

In [ ]:
#Read countries
fname_cntrs = dir_ctrs + 'Country codes 3 letters.xlsx'
data_cntrs  = pd.read_excel(fname_cntrs, sheet_name=0, header=None, index_col=0)

#Define country list for EU27 + UK
ctrs_EU = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR']

#Select countries for plotting
countries_sel = ['USA', 'RUS', 'EU27_UK', 'CAN', 'CHN', 'BRA', 'IDN', 'COD']


## BLUE forest areas

In [ ]:
#Read BLUE forest area
fname_prim = dir_area + 'BLUE_forest_area_primary_countries-ISOcode_vRemapArea_1991-2020.xlsx'
fname_secd = dir_area + 'BLUE_forest_area_secondary_countries-ISOcode_vRemapArea_1991-2020.xlsx'
data_prim_BLUE = pd.read_excel(fname_prim, header=0, index_col=0)
data_secd_BLUE = pd.read_excel(fname_secd, header=0, index_col=0)

#Add EU27&UK forest area
data_prim_BLUE['EU27_UK'] = data_prim_BLUE[ctrs_EU].sum(axis=1)
data_secd_BLUE['EU27_UK'] = data_secd_BLUE[ctrs_EU].sum(axis=1)

#Convert to Mio ha
data_prim_BLUE = data_prim_BLUE / 1e6 
data_secd_BLUE = data_secd_BLUE / 1e6


## FAO forest areas

In [ ]:
#Read data
fname_FAO = dir_area + 'FAO forest area.xlsx'
data_FAO  = pd.read_excel(fname_FAO, header=0, index_col=0)

#Replace name of EU countries by ISO names
for ctr_short, ctr_long in data_cntrs.iterrows():
    if ctr_long.values[0] in data_FAO.columns:
        data_FAO = data_FAO.rename(columns={ctr_long.values[0]: ctr_short})
        
#Add EU27&UK forest area
data_FAO['EU27_UK'] = data_FAO[ctrs_EU].sum(axis=1)

#Convert to Mio ha
data_FAO = data_FAO / 1e3

#Calculate area of secondary forest
data_FAO.loc['area secondary 2000'] = data_FAO.loc['area 2000'] - data_FAO.loc['area primary 2000']
data_FAO.loc['area secondary 2015'] = data_FAO.loc['area 2015'] - data_FAO.loc['area primary 2015']


## NGHGI forest areas

In [ ]:
#Read data
fname_NGHGI = dir_area + 'NGHGI forest area.xlsx'
data_NGHGI  = pd.read_excel(fname_NGHGI, header=0, index_col=0)

#Select area data
data_NGHGI = data_NGHGI.iloc[0:3, :]

#Convert to Mio ha
data_NGHGI = data_NGHGI / 1e3


## Non-intact forest areas

In [ ]:
#Read data
fname_HanPot = dir_area + 'HansenPotapov_forest_area_intact_non-intact_2013.xlsx'
data_HanPot  = pd.read_excel(fname_HanPot, header=0, index_col=0)

#Convert m2 to Mio ha
data_HanPot = data_HanPot / 1e4 / 1e6

#Add EU27&UK forest area
data_HanPot['EU27_UK'] = data_HanPot[ctrs_EU].sum(axis=1)


## Plot preparation

In [ ]:
#Colors
col_prim = 'forestgreen'
col_sec  = 'limegreen'
col_tot  = '#154406'

#Width of bars
wid = 0.175


## Plot

In [ ]:
#Create figure and add second axis
fig, ax = plt.subplots(1, 1, figsize=(18 * len(countries_sel)/10, 5))

#Loop over countries
ctrs = []
for i, country in enumerate(countries_sel):
    
    #Read data
    BLUE_prim = data_prim_BLUE[country].loc[slice(2011, 2015)].mean()
    BLUE_sec  = data_secd_BLUE[country].loc[slice(2011, 2015)].mean()
    BLUE_tot  = BLUE_prim + BLUE_sec
    NGHGI_man  = data_NGHGI[country]['forest managed area']
    NGHGI_uman = data_NGHGI[country]['forest unmanaged area']
    NGHGI_tot  = data_NGHGI[country]['total forest area']
    FAO_prim = data_FAO[country]['area primary 2015']
    FAO_sec  = data_FAO[country]['area secondary 2015']
    FAO_tot  = FAO_prim + FAO_sec
    HanPot_prim = data_HanPot[country]['intact']
    HanPot_sec  = data_HanPot[country]['non-intact']
    HanPot_tot  = HanPot_prim + HanPot_sec    
    
    #Define x-values
    x1 = i + 0.2
    x2 = i + 0.4
    x3 = i + 0.6
    x4 = i + 0.8
    
    #Plot Hansen/Potapov data
    ax.bar(x1, HanPot_sec, width=wid, color=col_sec)
    ax.bar(x1, HanPot_prim, bottom=HanPot_sec, width=wid, color=col_prim)    
    
    
    #Plot UNFCCC data
    if np.isnan(NGHGI_man) or np.isnan(NGHGI_uman):
        p1 = ax.bar(x2, NGHGI_tot, width=wid, color=col_tot)
    else:
        p2 = ax.bar(x2, NGHGI_man, width=wid, color=col_sec)
        p3 = ax.bar(x2, NGHGI_uman, bottom=NGHGI_man, width=wid, color=col_prim)
    
    #Plot FAO data
    ax.bar(x3, FAO_sec, width=wid, color=col_sec)
    ax.bar(x3, FAO_prim, bottom=FAO_sec, width=wid, color=col_prim)
    
    #Plot BLUE data
    ax.bar(x4, BLUE_sec, width=wid, color=col_sec)
    ax.bar(x4, BLUE_prim, bottom=BLUE_sec, width=wid, color=col_prim)

    
    #Add hatching for forest area affected by disturbance (according to Canada's UNFCCC reports)
    if country=='CAN':
        for_disturbed = 55
        ax.bar(x2, for_disturbed, bottom=NGHGI_man-for_disturbed, width=wid, facecolor='none', linewidth=0, edgecolor='forestgreen', hatch='xxx')
        
    #Make stripes in background
    if np.mod(i, 2)==0:
        ax.axvspan(i, i + 1, facecolor='black', alpha=0.05)
            
    #Write names of data
    if i==0:
        ax.text(x1, 20, 'Han&Pot', ha='center', va='bottom', color='w', rotation=90, fontsize=13)
        ax.text(x2, 20, 'UNFCCC', ha='center', va='bottom', color='w', rotation=90, fontsize=13)
        ax.text(x3, 20, 'FAO',   ha='center', va='bottom', color='w', rotation=90, fontsize=13)
        ax.text(x4, 20, 'BLUE', ha='center', va='bottom', color='w', rotation=90, fontsize=13)

            
    #Define country names
    if country=='USA':        ctr_name = country
    elif country=='RUS':      ctr_name = 'Russia'
    elif country=='COD':      ctr_name = 'DR Congo'
    elif country=='TZA':      ctr_name = 'Tanzania'
    elif country=='CAF':      ctr_name = 'Cent. Afr. Rep.'
    elif country=='EU27_UK':  ctr_name = 'EU27&UK'
    else:                     ctr_name = data_cntrs.loc[country].item()
    ctrs.append(ctr_name)
    
#Set x-ticks and x-tick labels
ax.set_xticks(np.arange(0.5, len(ctrs)))
ax.set_xticklabels(ctrs, rotation=0, ha='center', fontsize=14)

#Set limits and y-label
ax.set_xlim([0, len(ctrs)])
ax.set_ylabel('Forest area / $10^6$ ha', fontsize=20, labelpad=15)    

#Set xticks and size of ticks
ax.tick_params(axis='y', labelcolor='k')
ax.tick_params(labelsize=15)
ax.tick_params(axis='x', which='major', pad=10)
ax.tick_params(axis="x", which="both", bottom=False, top=False)

#Legend
ax.legend([p2[0], p3[0], p1[0]], ['managed', 'unmanaged', 'total'], fontsize=18, frameon = False, loc=8, ncol=3, bbox_to_anchor=(0.5, -0.38))

#Save figure
fig.savefig(dir_fig + 'Fig3_forest_areas.png', dpi=300, bbox_inches='tight')
fig.savefig(dir_fig + 'Fig3_forest_areas.pdf', bbox_inches='tight')
